In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as fn
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

import pytorch_lightning as pl

import numpy as np



c:\Users\patbo\.conda\envs\theoryML\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
class BasicLSTM(pl.LightningModule):

    def __init__(self, num_feat, num_hiddens, num_out):
        '''
        num_feat - number of features input into the model
        '''
        super().__init__()
        self.num_feat = num_feat
        self.num_hiddens = num_hiddens
        self.num_out = num_out

        shape_w1 = (num_hiddens,num_feat)
        shape_w2 = (num_hiddens,num_hiddens)
        
        mean = 0.0
        std = 1.0



        #the forget gate weights and bias 
        #(for f_t = sig(wf1 x_t + wf2 h_{t-1} + b_f))
        self.wf1 ,self.wf2, self.bf = self.initWeights(shape_w1, shape_w2, mean, std)

        #the input gate weights and bias
        #(for i_t = sig(wi1 x_t + wi2 h_{t-1} + b_i))
        self.wi1, self.wi2, self.bi = self.initWeights(shape_w1, shape_w2, mean, std) 
        #the output gate weights and bias
        #(for o_t = sig(wo1 x_t + wo2 h_{t-1} + b_o))
        self.wo1, self.wo2, self.bo = self.initWeights(shape_w1, shape_w2, mean, std)
        #the candidate context weights and bias
        #(for c^'_t = sig(wcc1 x_t + wcc2 h_{t-1} + bc_c))
        self.wcc1, self.wcc2, self.bcc = self.initWeights(shape_w1, shape_w2, mean, std)

        self.whq = nn.Parameter(torch.normal(mean=mean,
                                             std=std,
                                             size=(num_hiddens, num_out)),
                                requires_grad=True,
                                )
        self.bq = nn.Parameter(torch.tensor(0.0),
                        requires_grad=True,
                        )

        print('wf1 \n', self.wf1.shape, '\n wf2 \n', self.wf2.shape, '\n bf \n', self.bf.shape)
        print('wi1 \n', self.wi1.shape, '\n wi2 \n', self.wi2.shape, '\n bi \n', self.bi.shape)
        print('wcc1 \n', self.wf1.shape, '\n wcc2 \n', self.wf2.shape, '\n bf \n', self.bcc.shape)
        print('wo1 \n', self.wo1.shape, '\n wo2 \n', self.wo2.shape, '\n bo \n', self.bo.shape)

    def initWeights(self, shape_w1, shape_w2, mean, std):
        w1 = nn.Parameter(torch.normal(mean=mean,std=std, size=shape_w1),
                                requires_grad=True,
                                )
        w2 = nn.Parameter(torch.normal(mean=mean,std=std,size=shape_w2),
                                requires_grad=True,
                                )
        bias = nn.Parameter(torch.zeros(self.num_hiddens),
                                requires_grad=True,
                                )
        return w1, w2, bias


    def unit(self, val_in, long_mem, short_mem):
        '''
        INPUTS:
            val_in - input into this step of the unit x_t

            long_mem - the long term memory at this step

            short_mem - the short term memory at this step
        OUTPUTS:

        
        '''
        val_in = val_in.float()
        print(val_in)
        print(self.wi1)
        print()

        i_t = torch.sigmoid((self.wi1@val_in)+(self.wi2@short_mem)+(self.bi))

        f_t = torch.sigmoid((self.wf1@val_in)+(self.wf2@short_mem)+(self.bf))
        
        o_t = torch.sigmoid((self.wo1@val_in)+(self.wo2@short_mem)+(self.bo))

        cc_t = torch.tanh((self.wcc1@val_in)+(short_mem@short_mem)+(self.bcc))

        # print('f_t ', f_t)
        # print('i_t ', i_t)
        # print('cc_t ', cc_t)
        # print('o_t ', o_t)

        #update the long term memory (c_t)
        c_t = (f_t*long_mem) + (i_t*cc_t)

        #update the short term memory (h_t)
        h_t = o_t*torch.tanh(c_t)
        # print('update_short_mem.shape, ', update_short_mem.shape)

        #for multiple inputs we not do a final layer that is just linear
        # Y = (update_short_mem@self.whq) + self.bq
        Y = h_t

        return [Y, c_t, h_t]

    def forward(self, input):
        '''
        in order case input should be an array with multiple inputs for the model.
        The columns are the features and the rows are the days
        '''
        n_seq = np.shape(input)[-1]

        # long_mem = torch.zeros(n_seq, requires_grad=False)
        # short_mem = torch.zeros(n_seq, requires_grad=False)
        long_mem = torch.zeros(self.num_hiddens)
        short_mem = torch.zeros(self.num_hiddens)
        
        for ii in range(0,n_seq-1):

            # long_mem[ii+1], short_mem[ii+1] = self.unit(input[ii], 
            #                                         long_mem[ii], 
            #                                         short_mem[ii],
            #                                         )
            # print(input[:,ii])
            Y, long_mem, short_mem = self.unit(input[:,ii], 
                                                    long_mem, 
                                                    short_mem,
                                                    )
        return Y


    def configure_optimizers(self):
        return Adam(self.parameters())

    def training_step(self, batch, batch_indx):

        input_i, label_i = batch
        output_i = self.forward(input_i[0])

        loss = (output_i - label_i)**2

        self.log("training loss", loss)

        return loss

    def test_step(self, batch, batch_idx):
        input_i, label_i = batch
        output_i = self.forward(input_i[0])

        test_loss = (output_i - label_i)**2

        self.log("test_loss", test_loss)


    def predict_step(self, batch, batch_idx, dataloader_idx=0):

        return self(batch)

In [3]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, r'D:\Documents\GitHub\itcs-8156\utils')

from preprocessing import (market_prepro,
                           lstm_timeseries_feat_and_targ,
                           
)

In [22]:
X_train.columns

Index(['Open', 'High', 'Low', 'Volume', 'High-Low', 'Close-Open', 'Day_date',
       'Month', 'Year', 'near_end_quarter', 'Day'],
      dtype='object')

In [24]:
# st = "Stocks"
st = "ETFs"

#Input stock name
sn = "aadr" 
f = r'D:\Desktop\College Spring 2023\machineLearning\project\coding\data'
X_train, X_test, T_train, T_test = market_prepro(f,st,sn,False,splitdata=True, stdzr='minmax')
print(X_train.shape)
print(T_train.shape)
# X,T = market_prepro(f,st,sn,False,splitdata=False)

dl_train, ds_train = lstm_timeseries_feat_and_targ(X_train[['Open','Low']], T_train, 4, 1,None)
dl_test, ds_test = lstm_timeseries_feat_and_targ(X_test['Open'], T_test, 4, 1,  None)
# dl_train, ds_train = lstm_timeseries_feat_and_targ(X_train['Open'], T_train, 3, 1, [ 'Year', 'Month' ,'Day_date', 'Day'])
# dl_test, ds_test = lstm_timeseries_feat_and_targ(X_test['Open'], T_test, 3, 1, [ 'Year', 'Month' ,'Day_date', 'Day'])

(1044, 11)
(1044,)


In [34]:
mdl_stock = BasicLSTM(num_feat=2, num_hiddens=2, num_out=1)


wf1 
 torch.Size([2, 2]) 
 wf2 
 torch.Size([2, 2]) 
 bf 
 torch.Size([2])
wi1 
 torch.Size([2, 2]) 
 wi2 
 torch.Size([2, 2]) 
 bi 
 torch.Size([2])
wcc1 
 torch.Size([2, 2]) 
 wcc2 
 torch.Size([2, 2]) 
 bf 
 torch.Size([2])
wo1 
 torch.Size([2, 2]) 
 wo2 
 torch.Size([2, 2]) 
 bo 
 torch.Size([2])


In [31]:
ds_train[0][0]

tensor([[0.0114, 0.0204, 0.0237, 0.0198],
        [0.0023, 0.0146, 0.0132, 0.0172]], dtype=torch.float64)

In [35]:
mdl_stock.forward(ds_train[0][0]).detach()

tensor([0.0114, 0.0023])
Parameter containing:
tensor([[-1.6435, -0.0734],
        [ 1.0014, -1.8881]], requires_grad=True)
tensor([0.0204, 0.0146])
Parameter containing:
tensor([[-1.6435, -0.0734],
        [ 1.0014, -1.8881]], requires_grad=True)
tensor([0.0237, 0.0132])
Parameter containing:
tensor([[-1.6435, -0.0734],
        [ 1.0014, -1.8881]], requires_grad=True)


tensor([ 0.0015, -0.0130])

In [5]:

trainer = pl.Trainer(max_epochs=10) # with default learning rate, 0.001 (this tiny learning rate makes learning slow)
trainer.fit(mdl_stock, train_dataloaders=dl_train)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\patbo\.conda\envs\theoryML\lib\site-packages\pytorch_lightning\trainer\setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
c:\Users\patbo\.conda\envs\theoryML\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(

  | Name | Type | Params
--------

Epoch 0:   0%|          | 0/1039 [00:00<?, ?it/s] 

ValueError: `self.log(training loss, tensor([[0.0011, 0.0002]], dtype=torch.float64))` was called, but the tensor must have a single element. You can try doing `self.log(training loss, tensor([[0.0011, 0.0002]], dtype=torch.float64).mean())`

In [ ]:
trainer.test(mdl_stock, dataloaders=dl_test)


In [ ]:
ds_test[0][0]

In [ ]:

def makepred(model, dataset):
    y = []
    t = []
    for ii in dataset:
        feat, lab = ii

        y.append(model.forward(feat).detach().numpy()[0])
        t.append(lab.numpy()[0])

    return y, t



In [ ]:
y_test, t_test = makepred(mdl_stock, ds_test)
y_train, t_train = makepred(mdl_stock, ds_train)

In [ ]:
print(y_test[0:10])
print(t_test[0:10])

In [ ]:
from sklearn.metrics import (mean_absolute_percentage_error,
                             r2_score,
)

# train_mape = mean_absolute_percentage_error(t_train, y_train)
# test_mape = mean_absolute_percentage_error(t_test, y_test)



# print('Training MAPE ', train_mape)
# print('Testing MAPE ', test_mape)

print('Training r2 ', r2_score(t_train, y_train) )
print('Testing r2 ', r2_score(t_test, y_test) )

In [ ]:
import matplotlib.pyplot as plt

def scatter_results(Y,T,title):
    plt.figure(figsize=(9,9))
    plt.scatter(Y,T)
    plt.xlabel('Model Prediction')
    plt.ylabel('True Value')
    plt.title(title)

In [ ]:
scatter_results(y_test, t_test, 'Testing')
scatter_results(y_train, t_train, 'Training')

In [ ]:
plt.scatter(range(len(t_test)),t_test)
plt.scatter(range(len(y_test)),y_test)